In [44]:
from pytrends.request import TrendReq
import pytrends as pt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pycountry
import plotly.graph_objects as go

In [2]:
pytrends = TrendReq(hl='en-US', tz=360)

kw_list = ['war', 'ukraine', 'russia'] # list of keywords to get data 

pytrends.build_payload(kw_list, cat=0, timeframe='today 3-m') 

In [74]:
#1 Interest over Time

data = pytrends.interest_over_time() 
data = data.reset_index()

In [75]:
data = data[(data['date'] >= '2022-02-01') & (data['date'] < '2022-04-03')]

In [76]:
data

date  war  ukraine  russia  isPartial
27 2022-02-01    3        1       2      False
28 2022-02-02    3        1       2      False
29 2022-02-03    3        1       2      False
30 2022-02-04    3        1       2      False
31 2022-02-05    3        1       2      False
..        ...  ...      ...     ...        ...
83 2022-03-29    4        8       4      False
84 2022-03-30    4        7       3      False
85 2022-03-31    4        6       3      False
86 2022-04-01    4        6       3      False
87 2022-04-02    4        6       3      False

[61 rows x 5 columns]

In [77]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['date'], y=data['ukraine'],
                    mode='lines+markers',
                    name='Ukraine'))
fig.add_trace(go.Scatter(x=data['date'], y=data['war'],
                    mode='lines+markers',
                    name='War'))
fig.add_trace(go.Scatter(x=data['date'], y=data['russia'],
                    mode='lines+markers', 
                    name='Russia', line_color='yellow'))

fig.show()

In [8]:
by_region = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=True, inc_geo_code=True)

by_region

geoCode  war  ukraine  russia
geoName                                     
Afghanistan         AF   27       37      36
Albania             AL   18       53      29
Algeria             DZ   24       63      13
American Samoa      AS   39       38      23
Andorra             AD   19       47      34
...                ...  ...      ...     ...
Western Sahara      EH   24       54      22
Yemen               YE   43       33      24
Zambia              ZM   22       41      37
Zimbabwe            ZW   19       44      37
Åland Islands       AX   23       42      35

[250 rows x 4 columns]

In [9]:
df = by_region.reset_index()

In [10]:
def iso_mapper(iso2):
    cntr = pycountry.countries.get(alpha_2=iso2)
    if cntr is None:
        return None
    return cntr.alpha_3

def flag_mapper(iso2):
    cntr = pycountry.countries.get(alpha_2=iso2)
    if cntr is None:
        return None
    return cntr.flag

In [11]:
df['iso3'] = df['geoCode'].apply(iso_mapper)
df['flag'] = df['geoCode'].apply(flag_mapper)
df

geoName geoCode  war  ukraine  russia iso3 flag
0       Afghanistan      AF   27       37      36  AFG   🇦🇫
1           Albania      AL   18       53      29  ALB   🇦🇱
2           Algeria      DZ   24       63      13  DZA   🇩🇿
3    American Samoa      AS   39       38      23  ASM   🇦🇸
4           Andorra      AD   19       47      34  AND   🇦🇩
..              ...     ...  ...      ...     ...  ...  ...
245  Western Sahara      EH   24       54      22  ESH   🇪🇭
246           Yemen      YE   43       33      24  YEM   🇾🇪
247          Zambia      ZM   22       41      37  ZMB   🇿🇲
248        Zimbabwe      ZW   19       44      37  ZWE   🇿🇼
249   Åland Islands      AX   23       42      35  ALA   🇦🇽

[250 rows x 7 columns]

In [12]:
fig = px.scatter_geo(df, locations='iso3', color='geoName',
                     hover_name='geoName', size='ukraine',
                     projection='orthographic', text='flag')
# fig.add_traces(
#     list(px.scatter_geo(df, locations='iso3', color='geoName',
#                      hover_name='geoName', size='war',
#                      projection='orthographic', text='flag').select_traces())
# )
fig.show()

In [26]:
df.sort_values(['ukraine'])[-20:]

geoName geoCode  war  ukraine  russia iso3 flag
93                     Guinea      GN    5       87       8  GIN   🇬🇳
83                    Germany      DE   10       87       3  DEU   🇩🇪
96                      Haiti      HT    7       87       6  HTI   🇭🇹
59              Côte d’Ivoire      CI    6       88       6  CIV   🇨🇮
51           Congo - Kinshasa      CD    5       88       7  COD   🇨🇩
50        Congo - Brazzaville      CG    6       89       5  COG   🇨🇬
78           French Polynesia      PF    8       89       3  PYF   🇵🇫
34               Burkina Faso      BF    5       89       6  BFA   🇧🇫
43                       Chad      TD    4       90       6  TCD   🇹🇩
49                    Comoros      KM    0       92       8  COM   🇰🇲
89                 Guadeloupe      GP    7       92       1  GLP   🇬🇵
76                     France      FR    6       92       2  FRA   🇫🇷
137                Martinique      MQ    7       92       1  MTQ   🇲🇶
77              French Guiana      GF    5       93       2  GUF   🇬🇫
183                   Réunion      RE    5       93       2  REU   🇷🇪
203            St. Barthélemy      BL    7       93       0  BLM   🇧🇱
208     St. Pierre & Miquelon      PM    6       94       0  SPM   🇵🇲
155             New Caledonia      NC    6       94       0  NCL   🇳🇨
42   Central African Republic      CF    1       96       3  CAF   🇨🇫
140                   Mayotte      YT    2       98       0  MYT   🇾🇹

In [42]:
fig = px.bar(df.sort_values(['ukraine'])[-50:], 
             color='geoName', 
             x='geoName', 
             y='ukraine', 
             title='Top 50 Interested Countries', 
             text='flag')
fig.show()

In [13]:
related_data = pytrends.related_queries()

In [14]:
related_data['ukraine']['top']

query  value
0         russia ukraine    100
1                 russia    100
2           ukraine news     85
3            war ukraine     28
4    news russia ukraine     25
5            russia news     25
6        russian ukraine     15
7         ukraine russie     15
8                 russie     15
9     russia ukraine war     14
10    russia and ukraine     14
11                 putin     13
12         putin ukraine     13
13           ukraine map     11
14      russland ukraine     11
15              russland     11
16          live ukraine     10
17      ukraine konflikt      9
18        ukraine latest      9
19        guerre ukraine      9
20         krieg ukraine      8
21          ukraine nato      7
22                  nato      7
23   ukraine latest news      7
24  russia ukraine putin      7

In [15]:
related_data['war']['top']

query  value
0                    the war    100
1                ukraine war     77
2                     russia     58
3                 russia war     58
4                  world war     51
5         war ukraine russia     38
6             russia ukraine     38
7                 god of war     29
8                  civil war     21
9                   cold war     16
10                  war news     16
11            war in ukraine     15
12               russian war     13
13               world war 2     12
14               world war 3     10
15                 total war      9
16          ukraine war news      9
17    russia and ukraine war      8
18       ukraine russian war      8
19                   war map      8
20                 war movie      8
21  putin russia ukraine war      8
22           war with russia      8
23            vladimir putin      8
24               world war 1      7

In [16]:
related_data['russia']['top']

query  value
0         russia ukraine    100
1                ukraine    100
2            russia news     36
3    russia ukraine news     25
4           ukraine news     25
5                    war     22
6             russia war     21
7            ukraine war     14
8     russia ukraine war     14
9     russia and ukraine     14
10        russia ucraina     13
11               ucraina     13
12          putin russia     12
13                 putin     11
14        russia ucrania     10
15               ucrania     10
16                guerra      8
17         guerra russia      8
18             us russia      7
19  russia ukraine putin      7
20            russia map      6
21                 china      6
22          china russia      6
23                  nato      5
24          russia today      5

In [17]:
fig = px.bar(related_data['ukraine']['top'], x='query', y='value', title='Related queries')
fig.show()

In [18]:
fig = px.bar(related_data['war']['top'], x='query', y='value', title='Related queries')
fig.show()